#Installing Dependencies

In [7]:
!pip install --upgrade onn

Requirement already up-to-date: onn in /usr/local/lib/python3.6/dist-packages (0.1.8)


In [8]:
!pip install -U imbalanced-learn

Requirement already up-to-date: imbalanced-learn in /usr/local/lib/python3.6/dist-packages (0.5.0)


##Importing Dependencies

In [34]:
from onn.OnlineNeuralNetwork import ONN
from onn.OnlineNeuralNetwork import ONN_THS
from sklearn.datasets import make_classification, make_circles
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from imblearn.datasets import make_imbalance
import numpy as np

Using TensorFlow backend.


## Initializing Network

In [0]:
onn_network = ONN(features_size=10, max_num_hidden_layers=5, qtd_neuron_per_hidden_layer=40, n_classes=10)

##Creating Fake Classification Dataset

In [0]:
X, Y = make_classification(n_samples=50000, n_features=10, n_informative=4, n_redundant=0, n_classes=10,
                           n_clusters_per_class=1, class_sep=3)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42, shuffle=True)

##Learning and predicting at the same time

In [25]:
for i in range(len(X_train)):
  onn_network.partial_fit(np.asarray([X_train[i, :]]), np.asarray([y_train[i]]))
  
  if i % 1000 == 0:
    predictions = onn_network.predict(X_test)
    print("Online Accuracy: {}".format(balanced_accuracy_score(y_test, predictions)))

Online Accuracy: 0.14337461746314914
Alpha:[0.83890086 0.04027476 0.04027476 0.04027476 0.04027476]
Training Loss: 1.296051
Online Accuracy: 0.9606844639729234
Alpha:[0.839583   0.04010423 0.04010423 0.04010423 0.04010423]
Training Loss: 0.42200255
Online Accuracy: 0.9587807527275185
Alpha:[0.8393374  0.04016563 0.04016563 0.04016563 0.04016563]
Training Loss: 0.3702089
Online Accuracy: 0.967292853621438
Alpha:[0.83981675 0.0400458  0.0400458  0.0400458  0.0400458 ]
Training Loss: 0.26317188
Online Accuracy: 0.9654591078178111
Alpha:[0.83988035 0.04002989 0.04002989 0.04002989 0.04002989]
Training Loss: 0.25282928
Online Accuracy: 0.9665110941847939
Alpha:[0.8398668  0.04003327 0.04003327 0.04003327 0.04003327]
Training Loss: 0.21191452
Online Accuracy: 0.9710897029043102
Alpha:[0.83997285 0.04000676 0.04000676 0.04000676 0.04000676]
Training Loss: 0.21445769
Online Accuracy: 0.9719590088952164
Alpha:[0.8397224  0.04006939 0.04006939 0.04006939 0.04006939]
Training Loss: 0.19888349
Onl

# Learning in batch with CUDA

In [26]:
onn_network = ONN(features_size=10, max_num_hidden_layers=5, qtd_neuron_per_hidden_layer=40, n_classes=10, batch_size=10, use_cuda=True)

Using CUDA :]


In [0]:
from torch.utils.data import Dataset, DataLoader
class Dataset(Dataset):

  def __init__(self, X, Y):
    self.X = X
    self.Y = Y

  def __len__(self):
      return len(self.X)

  def __getitem__(self, idx):
      X = self.X[idx],
      Y = self.Y[idx]

      return X, Y
    
transformed_dataset = Dataset(X_train, y_train)
dataloader = DataLoader(transformed_dataset, batch_size=10,shuffle=True, num_workers=1)

In [28]:
for local_X, local_y in dataloader: 
  onn_network.partial_fit(np.squeeze(torch.stack(local_X).numpy()), local_y.numpy())

Alpha:[0.83902884 0.04024278 0.04024278 0.04024278 0.04024278]
Training Loss: 1.577592
Alpha:[0.8394673  0.04013318 0.04013318 0.04013318 0.04013318]
Training Loss: 0.5495532
Alpha:[0.8397412 0.0400647 0.0400647 0.0400647 0.0400647]
Training Loss: 0.40501082


In [29]:
predictions = onn_network.predict(X_test)
print("Accuracy: {}".format(balanced_accuracy_score(y_test, predictions)))

Accuracy: 0.9517950352535276


#Using contextual bandit - ONN_THS

In this example the ONN acts like a contextual bandits a reinforcement learning algorithm type. 

In [0]:
X_linear, Y_linear = make_classification(n_samples=10000, n_features=2, n_informative=2, n_redundant=0, n_classes=2, n_clusters_per_class=1, class_sep=200, shuffle=True)
X_non_linear, Y_non_linear = make_circles(n_samples=10000, noise=0.1, factor=0.3, shuffle=True)
X_linear_2, Y_linear_2 = make_classification(n_samples=10000, n_features=2, n_informative=2, n_redundant=0, n_classes=2, n_clusters_per_class=1, class_sep=200, shuffle=True)

X_linear_train = X_linear[:5000]
Y_linear_train = Y_linear[:5000]

X_linear_test = X_linear[5000:]
Y_linear_test = Y_linear[5000:]

X_non_linear_train = X_non_linear[:5000]
Y_non_linear_train = Y_non_linear[:5000]

X_non_linear_test = X_non_linear[5000:]
Y_non_linear_test = Y_non_linear[5000:]

X_linear_train_2 = X_linear_2[:5000]
Y_linear_train_2 = Y_linear_2[:5000]

X_linear_test_2 = X_linear_2[5000:]
Y_linear_test_2 = Y_linear_2[5000:]

In [0]:
gp = ONN_THS(2, 5, 100, 2)

In [32]:
for epoch in range(5):

    for i in range(len(X_linear_train)):
        x = np.asarray([X_linear_train[i, :]])
        y = np.asarray([Y_linear_train[i]])

        arm, exp = gp.predict(x)
        
        if arm == y[0]:  
          gp.partial_fit(x, y, exp)
          
        if i % 2000 == 1999:
          pred = []
          print("======================================================")
          for i in range(len(X_linear_test)):  
            pred.append(gp.predict(np.asarray([X_linear_test[i, :]]))[0])
          print("Accuracy: " + str(balanced_accuracy_score(Y_linear_test, pred)))
          print("======================================================")

print('Finished Training')

Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 7.708073e-07
Accuracy: 0.9320135479448636
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 0.0
Alpha:[0.66480255 0.08414558 0.08314086 0.08373952 0.0841715 ]
Training Loss: 1.4113159
Accuracy: 0.7803689523703463
Alpha:[0.4597675  0.1319707  0.13887632 0.13379806 0.13558747]
Training Loss: 2.1171865
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 0.8254292
Accuracy: 0.8239034376118035
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 1.9921635
Alpha:[0.09616726 0.22576231 0.22466676 0.22605874 0.2273449 ]
Training Loss: 1.2512604
Accuracy: 0.9194272332380344
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 0.0
Alpha:[0.7483906  0.06154997 0.0657564  0.0615586  0.06274442]
Training Loss: 1.2462758
Accuracy: 0.9089153835864292
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 1.1584326
Alpha:[0.83999

In [33]:
for epoch in range(5):

    for i in range(len(X_non_linear_train)):
        x = np.asarray([X_non_linear_train[i, :]])
        y = np.asarray([Y_non_linear_train[i]])

        arm, exp = gp.predict(x)
        
        if arm == y[0]:  
          gp.partial_fit(x, y, exp)
          
        if i % 2000 == 1999:
          pred = []
          print("======================================================")
          for i in range(len(X_linear_test)):  
            pred.append(gp.predict(np.asarray([X_non_linear_test[i, :]]))[0])
          print("Accuracy: " + str(balanced_accuracy_score(Y_non_linear_test, pred)))
          print("======================================================")

print('Finished Training')

Alpha:[0.7324713  0.06278172 0.07632934 0.0627818  0.06563586]
Training Loss: 0.565434
Accuracy: 0.4978415196546431
Alpha:[0.83837765 0.04006734 0.04134858 0.04006734 0.04013911]
Training Loss: 0.32934776
Accuracy: 0.6431012228961956
Alpha:[0.83944225 0.04013941 0.04013941 0.04013941 0.04013941]
Training Loss: 0.27433354
Alpha:[0.83982235 0.04004439 0.04004439 0.04004439 0.04004439]
Training Loss: 0.2834035
Alpha:[0.83984315 0.04003919 0.04003919 0.04003919 0.04003919]
Training Loss: 0.2099059
Accuracy: 0.7659370825499332
Alpha:[0.8395842  0.04010393 0.04010393 0.04010393 0.04010393]
Training Loss: 0.21292916
Accuracy: 0.9151972664315626
Alpha:[0.83984536 0.04003864 0.04003864 0.04003864 0.04003864]
Training Loss: 0.19620447
Alpha:[0.8398821  0.04002946 0.04002946 0.04002946 0.04002946]
Training Loss: 0.16505778
Alpha:[0.83968943 0.04007762 0.04007762 0.04007762 0.04007762]
Training Loss: 0.14461489
Accuracy: 0.9197904671664747
Alpha:[0.8399311  0.04001722 0.04001722 0.04001722 0.04001

# Imbalanced Dataset

In [0]:
X, Y = make_classification(n_samples=110000, n_features=20, n_classes=10, n_informative=8, n_redundant=0, n_clusters_per_class=1, class_sep=900)

In [0]:
X_t, Y_t = make_imbalance(X, Y, sampling_strategy={0: 800, 1: 5000, 2: 10000, 3: 10000, 4: 1000, 5: 1000, 6: 500, 7: 10000, 8: 5000, 9:5000})

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(X_t, Y_t, test_size=0.2)

In [0]:
gp = ONN_THS(20, 5, 100, 10)

In [49]:
for i in range(len(X_train)):
    x = np.asarray([X_train[i, :]])
    y = np.asarray([y_train[i]])

    arm, exp = gp.predict(x)

    if arm == y[0]:  
      gp.partial_fit(x, y, exp)

    if i % 2000 == 1999:
      pred = []
      print("======================================================")
      for i in range(len(X_test)):  
        pred.append(gp.predict(np.asarray([X_test[i, :]]))[0])
      print("Accuracy: " + str(balanced_accuracy_score(y_test, pred)))
      print("======================================================")

print('Finished Training')

Accuracy: 0.4353221256829899
Alpha:[0.06869241 0.16809322 0.3725306  0.17724589 0.21343793]
Training Loss: 9.325638
Accuracy: 0.4386085948153132
Alpha:[0.11643752 0.20428126 0.23810413 0.20844749 0.23272966]
Training Loss: 11.25701
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 2.6811903
Accuracy: 0.6928950497293518
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 10.146265
Accuracy: 0.6838257488564606
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 8.005989
Accuracy: 0.6115088279471166
Alpha:[0.79145914 0.05235159 0.05036381 0.05197832 0.05384716]
Training Loss: 3.7819355
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 8.093227
Accuracy: 0.7768358673631839
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 0.0
Accuracy: 0.7728323692342458
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 0.0
Alpha:[0.8399999 0.04      0.04      0.04      0.04

In [0]:
gp = ONN_THS(20, 5, 100, 10)

In [51]:
for i in range(len(X_train_t)):
    x = np.asarray([X_train_t[i, :]])
    y = np.asarray([y_train_t[i]])

    arm, exp = gp.predict(x)

    if arm == y[0]:  
      gp.partial_fit(x, y, exp)

    if i % 2000 == 1999:
      pred = []
      print("======================================================")
      for i in range(len(X_test_t)):  
        pred.append(gp.predict(np.asarray([X_test_t[i, :]]))[0])
      print("Accuracy: " + str(balanced_accuracy_score(y_test_t, pred)))
      print("======================================================")

print('Finished Training')

Alpha:[0.31641227 0.12936862 0.17681654 0.18663488 0.19076766]
Training Loss: 4.9937906
Accuracy: 0.5121877357778365
Alpha:[0.11614554 0.1702754  0.2646138  0.1885041  0.2604611 ]
Training Loss: 9.42681
Accuracy: 0.60000532007458
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 3.760187
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 0.0
Accuracy: 0.6370007505514537
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 0.0
Accuracy: 0.6198465258532846
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 0.0
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 1.579348
Accuracy: 0.5121875529066693
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 9.70304
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 0.0
Accuracy: 0.6330198059531864
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 10.2148285
Accuracy: 0.70849